In [9]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [76]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [77]:
%run -i ../../core.py

In [78]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [79]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.tensor(np.median(dummy_data))
# print(int(len(dummy_data)/2))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data)[int(len(dummy_data)/2)])


In [80]:
print("dummy output: ", dummy_theory_output)
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))

        # doesnt support mod if not in conditional
       
        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len
        # print("more: ", more_cons)


        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))

        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

dummy output:  tensor(5.4000, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_55637/3411461829.py:20: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):


In [81]:
# prover calculates settings, send to verifier

theory_output = torch.tensor(np.median(data))
lower_to_median = torch.tensor(np.sort(data)[int(len(data)/2)-1])
upper_to_median = torch.tensor(np.sort(data)[int(len(data)/2)])
print("Theory_output: ", theory_output)
print("lower: ", lower_to_median)
print("upper: ", upper_to_median)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # since within 1%, we regard as same value
        count_less = torch.sum((X < 0.99*self.w).double())
        count_equal = torch.sum((torch.abs(X-self.w)<=torch.abs(0.01*self.w)).double())
        len = X.size()[1]
        half_len = torch.floor(torch.div(len, 2))

        # doesnt support mod if not in conditional
       
        # For count_equal > 0
        if(len%2==0):
            is_odd = 0
        else:
            is_odd = 1
        less_cons = count_less<half_len+is_odd
        more_cons = count_less+count_equal>half_len
        # print("more: ", more_cons)


        # For count_equal == 0
        lower_exist = torch.sum((torch.abs(X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((X<0.99*self.upper).double())==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))
        
        return (median_in_cons*(count_equal!=0)+median_out_cons*(count_equal ==0), self.w)

 

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

Theory_output:  tensor(49.5500, dtype=torch.float64)
lower:  tensor(49.3000, dtype=torch.float64)
upper:  tensor(49.8000, dtype=torch.float64)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_55637/1656333733.py:26: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if(len%2==0):


==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-6278,6246],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":12047,"total_const_size":1819,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [82]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2
spawning module 0


Time setup: 1.626837968826294 seconds
Theory output:  tensor(49.5500, dtype=torch.float64)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 49.546875
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[3241277322567916088, 15136847618162288068, 13095027618627327699, 3297383697173931210], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15311375669850521209, 11431854252320932136, 7779939630960000938, 229532113155256883]]], 'proof': '2ea26df955ea6b0d97597c56665130d7fc8198f98f33fc8fbf1bcd29ed76955f2dcacc1bbc43d7970b679a76f573405629306aa4e7e2abb06c2bbd14779177eb131565bcb343a4061fbc45ec88961f3822478685ddcdcd69a035e03ead583a131877261a1fb448c39b177d4dce7c48c679a524ff011eccf2c564d7809a4bc5f91cac961aa501b97f7c2a33034f470ba46490578a363266c56672842069632e0427dccc2bd05398c2870142e07c2fc9125a2fba8b0f655b3543ac8d8fcbc2ea370892af50d5f5d6576c3e2703dacd3f1cf32ec27cd338088d890da14141a0c1b603f4bbaf9f615140c9b5de0efc6944d59b515356c496a9eeaa686ecf81e7c1da06cac8666550215a3bc01aaf13a5b50f5e86913bea51eee337d70ab64f24afb82f13a11e326ae1f97ddb8213f5e251c984dcb1571f0c3353bfc3583d8444dbed001d15b83d9efef6ad9bf001bbb21f995c42e138cb8da1be3efbbbddb4dd80

In [83]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[3241277322567916088, 15136847618162288068, 13095027618627327699, 3297383697173931210], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [15311375669850521209, 11431854252320932136, 7779939630960000938, 229532113155256883]]]
proof boolean:  1.0
proof result 1 : 49.546875
verified
